In [4]:
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

def ollama_chat_client():
    return OpenAIChatClient(
        api_key="ollama",
        model_id="llama3.2:1b",
        base_url="http://localhost:11434/v1"  # la URL de tu servidor Ollama
    )



### Agents

In [5]:
agent = ChatAgent(
    chat_client=ollama_chat_client(),
    instructions="You are a helpful assistant.",
    name="LocalOllamaAgent"
)

agent_response = await agent.run("What model are you?")
agent_response.text


'I\'m an artificial intelligence model, and I don\'t have a specific "model" like a computer game or software application. I\'m a type of natural language processing (NLP) model designed to understand and generate human-like text. My underlying technology is based on advanced algorithms and large datasets, but I don\'t have a specific hardware or software platform that sets me apart from other language models. I exist solely as a digital entity, running on servers and responding to inputs from users like you!'

In [23]:
from datetime import datetime
from agent_framework import ai_function

@ai_function(name="time_tool", description="Retrieves the actual time")
def get_time() -> str:
    time = datetime.now()
    return f"The hour is {time.hour}:{time.minute}"

In [25]:
time_agent = ChatAgent(
    chat_client=ollama_chat_client(),
    instructions="You are a helpful assistant.",
    name="LocalOllamaAgent",
    tools=get_time
)

In [26]:
time_agent_response = await time_agent.run("What hour it is?")
time_agent_response.text

'The hour is 7:02 PM. Would you like to know the current date, weather, or a random fact?'

### Workflows

In [27]:
import asyncio
from typing_extensions import Never
from agent_framework import WorkflowBuilder, WorkflowContext, WorkflowOutputEvent, executor, Executor, handler

In [28]:
class UpperCase(Executor):
    def __init__(self, id: str):
        super().__init__(id=id)

    @handler
    async def to_upper_case(self, text: str, ctx: WorkflowContext[str]) -> None:
        result = text.upper()
        await ctx.send_message(result)

In [29]:
@executor(id="reverse_text_executor")
async def reverse_text(text: str, ctx: WorkflowContext[Never, str]) -> None:
    result = text[::-1]
    await ctx.yield_output(result)

In [30]:
upper_case = UpperCase(id="upper_case_executor")

workflow = (
    WorkflowBuilder()
    .add_edge(upper_case, reverse_text)
    .set_start_executor(upper_case)
    .build()
)

In [31]:
async def main():
    async for event in workflow.run_stream("hello world"):
        print(f"Event: {event}")
        if isinstance(event, WorkflowOutputEvent):
            print(f"Workflow completed with result: {event.data}")


In [32]:
await main()

Event: WorkflowStartedEvent(origin=WorkflowEventSource.FRAMEWORK, data=None)
Event: WorkflowStatusEvent(state=WorkflowRunState.IN_PROGRESS, data=None, origin=WorkflowEventSource.FRAMEWORK)
Event: ExecutorInvokedEvent(executor_id=upper_case_executor, data=None)
Event: ExecutorCompletedEvent(executor_id=upper_case_executor, data=None)
Event: ExecutorInvokedEvent(executor_id=reverse_text_executor, data=None)
Event: WorkflowOutputEvent(data=DLROW OLLEH, source_executor_id=reverse_text_executor)
Workflow completed with result: DLROW OLLEH
Event: ExecutorCompletedEvent(executor_id=reverse_text_executor, data=None)
Event: WorkflowStatusEvent(state=WorkflowRunState.IDLE, data=None, origin=WorkflowEventSource.FRAMEWORK)


#### Workflow with agents

In [47]:
writer = ChatAgent(
    instructions=(
        "Eres un gran escritor especializado en futbol"
    ),
    name="writer",
    chat_client=ollama_chat_client()
)

reviewer = ChatAgent(
    instructions=(
        "Eres un critico de futbol que corrige y edita textos"
    ),
    name="reviewer",
    chat_client=ollama_chat_client()
)

In [48]:
from agent_framework import AgentRunUpdateEvent, WorkflowBuilder, WorkflowOutputEvent

workflow = WorkflowBuilder().set_start_executor(writer).add_edge(writer, reviewer).build()

In [49]:
events = workflow.run_stream("Crea una publicacion acerca de Lionel Messi")

async for event in events:
    if isinstance(event, AgentRunUpdateEvent):
        print(event.data, end="", flush=True)

    elif isinstance(event, WorkflowOutputEvent):
        print("\n===== Final output =====")
        print(event.data)

**El Rey del Fútbol: La Legenda de Lionel Messi**

 ¿Quién es el hombre más famoso en el mundo del deporte? ¡Es nada menos que el legendaro Lionel Messi! Con 7 títulos mundiales, 10 galones de Oro Olímpicos y el balón de oro como mayor anotador de la historia, Lionel sigue siendo una figura imbatible en el fútbol.

 **Un Hombre con una Vida de Suerte**

 Nadie sabía si Messi iba a ser un jugador del FC Barcelona o de AC Milan. La suerte pareció estar desde el primer día que se incorporó al equipo. Pero lo que no sabían era que tenía una gran suerte, una confianza en sí mismo y una pasión incendiosa por el fútbol.

 **La Transmisione del Suor**

 En 2004, Messi debutó como jugador de liga con el Newell's Old Christians, pero pronto se uniría alFC Barcelona. Durante su etapa con los catalanes, Messi convirtió a la nación del fútbol argentino en una fuente de orgullo nacional. Su talento, destreza y habilidad para marcar goles le valieron tanto respeto como admiración del público.

 **El 